In [2]:
import sys
import os
from elasticsearch import Elasticsearch
from zipfile import ZipFile
import re
import pandas as pd
import json
import requests
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import nltk
import pickle
import pytrec_eval
nltk.download('stopwords')
nltk.download('punkt')
from elasticsearch import NotFoundError

es = Elasticsearch()

[nltk_data] Downloading package stopwords to /home/katya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/katya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
input_dir = 'data/docs'
output_dir = 'b0k5'

In [4]:
def strip_punct(s):
    s = re.sub('[^A-Za-z0-9]', ' ', s)
    s = s.lower()
    return " ".join(s.split())

## Create new index, adjust BM25-parameters

In [104]:
request_body = {
    "settings" : {
	        "number_of_shards": 1,
	        "number_of_replicas": 0,
        "similarity": {
      "default": { 
        "type": "BM25",
        "b":0,
        "k1":5
      }
    }
	    }
	}

es.indices.create(index = 'b0k5_index', body = request_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'b0k5_index'}

In [70]:
bulk_data = []
   
count = 0
c=0
t = 1
url = 'https://demo.webis.de/targer-api/classifyCombo'
headers = {'accept': 'application/json', 'Content-Type': 'text/plain'}
#extract docs from zip-files
for filename in os.listdir(input_dir):
    if filename.endswith(".zip"):
        with ZipFile((input_dir+"/{}").format(filename), 'r') as zip:
            zip.extractall(input_dir)
            
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        name = re.split('_|\.', filename)
        num = next(obj for obj in name if obj.isdigit())
        with open((input_dir+"/{}").format(filename), "r") as f:
            topic = json.load(f)
            topic['title'] = strip_punct(topic['title']).lower()
            print(topic['title'])
            print(t)
            t+=1
            for n in topic["documents"]:
                print(len(n))
                #print(count)
                if count<10:
                    count+=1
                    for doc in n:
                        try:
                            doc['lem'] = word_tokenize(strip_punct(doc['document']).lower())
                            tokens_without_sw = [word for word in doc['lem'] if not word in stopwords.words()]
                            lemmatizer = WordNetLemmatizer()
                            lemmatized_output_0 = ([lemmatizer.lemmatize(w,pos="n") for w in tokens_without_sw])
                            doc['lem'] = ' '.join(([lemmatizer.lemmatize(w,pos="v") for w in lemmatized_output_0]))
                            doc['lem'] = doc['lem'].rstrip()
                            
                            title_lem = strip_punct(doc['title'].lower().rstrip())
                            title_lem = word_tokenize(title_lem)
                            tokens_without_sw = [word for word in title_lem if not word in stopwords.words()]
                            lemmatizer = WordNetLemmatizer()
                            lemmatized_output_0 = ([lemmatizer.lemmatize(w,pos="n") for w in tokens_without_sw])
                            title_lem = ' '.join(([lemmatizer.lemmatize(w,pos="v") for w in lemmatized_output_0]))
                            
                            doc_raw = doc['document'].rstrip()
                            res = requests.post(url, json=doc_raw)
                            res = res.json()
                            args = []
                            for x in res:
                                arg = []
                                for el in x:
                                    if el['label']!='O' and float(el['prob'])>0.99:
                                        arg.append(el['token'])
                                s = ' '.join(arg)
                                args.append(s)
                            arguments = ' '.join(args)
                            arguments = word_tokenize(strip_punct(arguments).lower())
                            tokens_without_sw = [word for word in arguments if not word in stopwords.words()]
                            lemmatizer = WordNetLemmatizer()
                            lemmatized_output_0 = ([lemmatizer.lemmatize(w,pos="n") for w in tokens_without_sw])
                            arguments = ' '.join(([lemmatizer.lemmatize(w,pos="v") for w in lemmatized_output_0]))
                            
                            data_dict = {'query': topic['title'],
                                    'title': doc['title'],
                                    'title_lem': title_lem,
                                    'num': num,
                                    'uuid': doc['uuid'],
                                    'score': doc['score'],
                                    'document': doc['document'],
                                    'lem': doc['lem'],
                                    'args': arguments}

                            op_dict = {
                                "index": {
                                "_index": 'b0k5_index',
                                "_type": 'doc',
                                "_id": doc['trec_id']
                                    }
                                }
                            bulk_data.append(op_dict)
                            bulk_data.append(data_dict)
    
                            c+=1
                        except KeyError:
                            pass
                else:
                    break
            count=0


which and has and more and caffeine and coffee and or and tea
1
10
10
10
10
10
10
10
10
10
10
10
which and one and is and better and an and electric and stove and or and a and gas and stove
2
10
10
10
10
10
10
10
10
10
10
10
why and is and linux and better and than and windows
3
10
10
10
10
10
10
10
10
10
10
10
which and company and has and a and larger and capitalization and apple and or and microsoft
4
10
10
10
10
10
10
10
10
10
10
10
what and is and better and for and the and environment and a and real and or and a and fake and christmas and tree
5
10
10
10
10
10
10
10
10
10
10
10
whose and salary and is and higher and basketball and or and soccer and players
6
10
10
10
10
10
10
10
10
10
10
10
which and is and better and pepsi and or and coke
7
10
10
10
10
10
10
10
10
10
10
10
which and is and healthiest and coffee and green and tea and or and black and tea and and and why
8
10
10
10
10
10
10
10
10
10
10
10
which and is and better and disneyland and or and disney and world
9
10
10
1

In [86]:
len(bulk_data)

10060

In [78]:
with open('bulk_data.txt', 'w') as f:
    for item in bulk_data:
        f.write("%s\n" % item)

In [ ]:
with open('bulk_data_json.txt', 'w') as outfile:
    json.dump(bulk_data, outfile)

In [100]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [122]:
bulks = chunks(bulk_data, 100)

In [119]:
bulks

<generator object chunks at 0x7f04a755d350>

In [123]:
for x in bulks:
    res = es.bulk(index = 'b0k5_index', body = x)


#es.search(body={"query": {"match_all": {}}}, index = 'example_index')
#es_indices.get_mapping(index = 'example_index')

In [5]:
res = es.get(index='b0k5_index',doc_type='doc',id='clueweb12-1214wb-88-29751')

/home/katya/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [6]:
res

{'_index': 'b0k5_index',
 '_type': 'doc',
 '_id': 'clueweb12-1214wb-88-29751',
 '_version': 1,
 '_seq_no': 3633,
 '_primary_term': 1,
 'found': True,
 '_source': {'query': 'what and is and the and difference and between and sex and and and love',
  'title': '<em>Difference</em> <em>Between</em> <em>Love</em> <em>And</em> <em>Sex</em> - Student.com Articles',
  'title_lem': 'difference love sex student article',
  'num': '1',
  'uuid': 'b0054765-4007-52c9-a7c4-70c15891b4cc',
  'score': 2406.7341,
  'document': 'Difference Between Love And Sex\nYou are here:\nStudent.com Articles Student Health & Sexuality Sexual Health Difference Between Love And Sex\nRelated Articles\n<p>Search, Compare, and Apply for student loans at <a href="http://www.simpletuition.com?brnd=studentcom" title="Simpletuition.com" target="_blank">Simpletuition.com</a>.</p>\nPaying For College\nSex VS. Love Love and sex are NOT the same thing. Love is an emotion or a feeling. There is no one definition of love because t

In [99]:
#es.indices.delete(index='b0k5_index')

{'acknowledged': True}

In [7]:
es.cat.count('b0k5_index', params={"format": "json"})

[{'epoch': '1613036530', 'timestamp': '09:42:10', 'count': '5017'}]

## Create different runs

In [8]:
f = open("data/topics_lemmatized.txt", 'rb')
queries_lem = pickle.load(f)
f.close()

In [9]:
def search(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1

    for i in q:
        for x in search_param['query']['bool']['should']:
            if type(i) == dict:
                for key in x['match']:
                    x['match'][key] = i['title']
            else:
                for key in x['match']:
                    x['match'][key] = i

    
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [10]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "title_lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_title', queries_lem, "b0k5_index")

In [11]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_doc', queries_lem, "b0k5_index")

In [12]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "args": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_arg', queries_lem, "b0k5_index")

In [13]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_arg', queries_lem, "b0k5_index")

In [14]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "lem": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc', queries_lem, "b0k5_index")

In [15]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_doc_arg', queries_lem, "b0k5_index")

In [16]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc_arg', queries_lem, "b0k5_index")

## Add synonyms

In [17]:
def search_extended(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1
#['bool']['should']
    for idx, row in q.iterrows():
            
            #for key in x['match']:
                #x['match'][key] = row['query']
        for x in search_param['query']['bool']['should']:
            for key in x["match"]:
                #print(key)
                if "boost" in x['match'][key]:
                    x['match'][key]["query"] = row['query']
                else:
                    x['match'][key] = row['syn']
        #print(search_param)
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [18]:
df_syn = pd.read_csv('data/q_for_syn.tsv', sep = '\t')

In [19]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_syn', df_syn, "b0k5_index")

In [20]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_syn', df_syn, "b0k5_index")

In [21]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_args_syn', df_syn, "b0k5_index")

In [22]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_arg_syn', df_syn, "b0k5_index")

In [23]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_syn', df_syn, "b0k5_index")

In [24]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_arg_syn', df_syn, "b0k5_index")

In [25]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_arg_syn', df_syn, "b0k5_index")

## Evaluate and compare results

In [26]:
def evaluate_run(run_path, qrel_path):
    run = pd.read_csv(run_path, sep = ' ', names = ['topic','Q0','id','rank','score','team'])

    qrel = pd.read_csv(qrel_path, sep = ' ', names = ['topic','Q0','id','relevance'])

    qrels = {}
    for i in range(1,51):
        qrels[str(i)] = {}
    
    for idx, row in qrel.iterrows():
        qrels[str(row['topic'])][row['id']] = row['relevance']
    
    runs = {}
    for i in range(1,51):
        runs[str(i)] = {}
    
    for idx, row in run.iterrows():
        runs[str(row['topic'])][row['id']] = row['score']

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrels,{'map_cut', 'ndcg_cut', 'recall', 'P'})

    res = evaluator.evaluate(runs)

    results = {}
    recall = {}
    pres = {}
    for key in res:
        results[key] = res[key]['ndcg_cut_10']
        recall[key] = res[key]['recall_10']
        pres[key] = res[key]['P_10']
        #print(res[key])
    
    filename = re.split('\.', sys.argv[1])
    with open((filename[0]+'_evaluated.json'), 'w') as f:
        json.dump(results, f)
        
    s = sum(results.values())/50
    r = sum(recall.values())/50
    p = sum(pres.values())/50
    if r+p != 0:
        f1 = (2*(r*p)/(r+p))
    else: 
        f1 = 0
    '''
    print("Average ndcg_cut_10: ", s)
    print("Average recall_10: ", r)
    print("Av. precision: ", p)
    print("F1-score: ",f1)
    '''    
    return s, r, p, f1


In [27]:
run = []
ndcg = []
rec = []
prec = []
f1s = []

for filename in os.listdir(output_dir):
    if filename.endswith(".txt"):
        #print(filename)
        s, r, p, f1 = evaluate_run((output_dir+"/{}").format(filename), 'touche2020-task2-relevance-withbaseline(1).qrels')
        run.append(filename)
        ndcg.append(s)
        rec.append(r)
        prec.append(p)
        f1s.append(f1)

In [28]:
df = pd.DataFrame({'run':run,'ndcg_cut10':ndcg, 'recall_10':rec, 'precision':prec, 'f1-score':f1s})

### All results

In [29]:
df.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
11,run_title_doc_arg_syn.txt,0.268515,0.168667,0.250,0.201433
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
4,run_doc_syn.txt,0.235968,0.156522,0.234,0.187575


In [30]:
df.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
11,run_title_doc_arg_syn.txt,0.268515,0.168667,0.250,0.201433
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
6,run_doc_arg.txt,0.226424,0.159938,0.234,0.190007


In [31]:
df.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
11,run_title_doc_arg_syn.txt,0.268515,0.168667,0.250,0.201433
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
4,run_doc_syn.txt,0.235968,0.156522,0.234,0.187575


In [32]:
df.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
11,run_title_doc_arg_syn.txt,0.268515,0.168667,0.250,0.201433
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
6,run_doc_arg.txt,0.226424,0.159938,0.234,0.190007


### Results without doc

In [33]:
df_p = df[~df.run.str.contains("doc")]

In [34]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
2,run_title.txt,0.246646,0.165060,0.250,0.198839
8,run_args_syn.txt,0.226296,0.151649,0.226,0.181505
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


In [35]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
8,run_args_syn.txt,0.226296,0.151649,0.226,0.181505
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


In [36]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
8,run_args_syn.txt,0.226296,0.151649,0.226,0.181505
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


In [37]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
1,run_title_arg_syn.txt,0.275510,0.174208,0.260,0.208628
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
8,run_args_syn.txt,0.226296,0.151649,0.226,0.181505
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


### Results for pipeline

In [38]:
df_p = df[~df.run.str.contains("arg")]
df_p = df_p[~df_p.run.str.contains("syn")]

In [39]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151


In [40]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151


In [41]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151


In [42]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151


### Results for synonyms

In [43]:
df_syn = df[~df.run.str.contains("arg")]

In [44]:
df_syn.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
4,run_doc_syn.txt,0.235968,0.156522,0.234,0.187575


In [45]:
df_syn.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
4,run_doc_syn.txt,0.235968,0.156522,0.234,0.187575


In [46]:
df_syn.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
4,run_doc_syn.txt,0.235968,0.156522,0.234,0.187575


In [47]:
df_syn.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
10,run_title_doc_syn.txt,0.338383,0.210363,0.316,0.252581
2,run_title.txt,0.246646,0.165060,0.250,0.198839
13,run_title_syn.txt,0.247895,0.161361,0.250,0.196130
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
4,run_doc_syn.txt,0.235968,0.156522,0.234,0.187575


### Results for arguments

In [48]:
df_arg = df[~df.run.str.contains("syn")]

In [49]:
df_arg.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
6,run_doc_arg.txt,0.226424,0.159938,0.234,0.190007
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


In [50]:
df_arg.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
6,run_doc_arg.txt,0.226424,0.159938,0.234,0.190007
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


In [51]:
df_arg.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
6,run_doc_arg.txt,0.226424,0.159938,0.234,0.190007
9,run_arg.txt,0.220213,0.150954,0.220,0.179051


In [52]:
df_arg.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.337165,0.211516,0.320,0.254687
12,run_title_arg.txt,0.287633,0.185470,0.272,0.220551
0,run_title_doc_arg.txt,0.275122,0.178376,0.264,0.212901
2,run_title.txt,0.246646,0.165060,0.250,0.198839
7,run_doc.txt,0.237161,0.161215,0.244,0.194151
6,run_doc_arg.txt,0.226424,0.159938,0.234,0.190007
9,run_arg.txt,0.220213,0.150954,0.220,0.179051
